In [72]:
import scipy
import numpy as np
import pandas as pd
import os
from PIL import Image

# MAT 167: Homework Assignment #1

### Problem 1.

(a)

In [106]:
# reading the tiff file after putting it into the current working directory
ind = os.listdir().index('plant.tiff')
file = os.listdir()[ind]

In [107]:
# using PIL package for reading and showing the .tiff file
img = Image.open(file)
img.show()

(b)

* Scipy was the best option for support with the dct function
* According to Scipy Docs: "For a single dimension array x, dct(x, norm='ortho') is equal to MATLAB dct(x)."
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.dct.html#scipy-fftpack-dct

In [108]:
y = scipy.fftpack.dct(img, norm='ortho')

# Indexing y at values where y is less than 10, and setting them equal to zero
y[abs(y[:])<10] = 0

# using numpy to reshape the matrix
y = np.reshape(y,(256,256))

In [109]:
# inverse dct used here
xx = scipy.fftpack.idct(y)

# In Python, to see image using PIL package, we need to conver the idct function results into
# a usable Image from our array which needs to be converted into a unsigned 8-bit integer
array = Image.fromarray(xx.astype('uint8')*255)
im = array.show()

(c)